# Crawl & Scrape any Website
#### Crawling Eenadu.net in this example
**Notebook Author**: Nirupam Purushothama

**Note:**
* This notebook crawls and scrapes a popular Telugu daily Eenadu.net
* Notebook should be modified to custom suit the crawl requirements of individual websites (i.e. based on link structures)
* Notebook provides code to download image files (you can easily extend this to video files as well)
* Notebook stores the crawled text in a json file. Provides the ability to link text to urls
* Notebook does not handle Captchas. Throttle the requests to not run into captchas. Else integrate with a Captcha decoder to auto answer captchas (this is work for a later date)

Happy crawling!!!

In [57]:
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
import shutil
from contextlib import closing
import hashlib
import json
import os.path
import time

In [58]:
def simple_get(url):
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
            
    except RequestException as e:
        log_error('Error during request to {0} : {1}'.format(url, str(e)))
        return None
    
def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
           and content_type is not None
           and content_type.find('html') > -1)

def log_error(e):
    print(e)

In [59]:
folder_loc = "./dump/"

def download_file(url):
    fname = url.split('/')[-1]
    local_filename = folder_loc + fname
    
    counter = 0
    
    while(os.path.isfile(local_filename) == True):
        local_filename = folder_loc + str(counter) + fname
        counter += 1
    
    with get(url, stream=True) as r:
        with open(local_filename, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return local_filename

In [136]:
def extract_img(raw_html, url=None):
    all_text = ""
    
    try:
        bs_html = BeautifulSoup(raw_html, 'html.parser')

        imgdiv = bs_html.find('section', attrs={'id': "content-smart"})

        if(imgdiv == None):
            return

        elements_to_grab = ['img']

        for ele in elements_to_grab:
            for p in imgdiv.select(ele):
                img_url = p["src"]
                if(img_url != None):
                    download_file(img_url)
                    # Use sleep to throttle the number of requests to server per minute/second. 
                    # There is a possibility of hitting a captcha if we do not throttle requests.
                    #time.sleep(0.25)
                
    except TypeError as e:
        print("")
        print("Exception: " + str(e) + " , URL: " + url)
        
    return None

def extract_text(raw_html, url=None):
    all_text = ""
    
    try:
        bs_html = BeautifulSoup(raw_html, 'html.parser')

        sectiondiv = bs_html.find('section', attrs={'id': "content-smart"})

        if(sectiondiv == None):
            return

        elements_to_grab = ['p']

        for ele in elements_to_grab:
            for p in sectiondiv.select(ele):
                p_text = p.text
                if(p_text != None):
                    all_text += p_text
                    
    except TypeError as e:
        print("")
        print("Exception: " + str(e) + " , URL: " + url)
        
    return all_text

def check_valid_url(url_string):
    isvalid = True
    
    if(url_string.find("eenadu.net") == -1 or url_string.find("eenadu.net") > 10):
        isvalid = False
        
    return isvalid

def make_url_useful(url_string):
    if(url_string.startswith("http") == True):
        return url_string
    elif(url_string.startswith("//") == True):
        return "https:"+url_string
    else:
        return "https://"+url_string

In [153]:
hash_to_url_map = {}
hash_to_text_map = {}

def crawl_scrape(url_proc_map, url_proc_stack, root_url):
    
    depth = 0
    deep = True
    extract_external_info = False
    data_dict = {}
    url_dict = {}
    
    # Extract only from 10 pages. You can increase the count to whatever you like
    n_steps = 10
    
    # Function handles only a open file handle
    
    while(len(url_proc_stack) > 0):
        depth += 1
        
        if(depth > n_steps):
            print("Reached crawl limit.")
            break
        
        full_url = url_proc_stack.pop()
        
        # Create the md5 hashcode for dictionary
        hash_code = hashlib.md5(full_url.encode())
        
        # Mark the URL as processed
        url_proc_map[hash_code.hexdigest()] = 1
        
        # Print status
        # print("Extracting ", full_url)
        print("=", end="")
        
        # Extract HTML
        rw_html = simple_get(full_url)
        
        # If there was an error reading that url then just continue
        if(rw_html == None):
            continue
        
        page_text = extract_text(rw_html, full_url)
        
        # Proceed to save
        if(page_text != None and page_text != ""):
            hash_to_url_map[hash_code.hexdigest()] = full_url
            hash_to_text_map[hash_code.hexdigest()] = page_text
        
        # Extract more urls from this text blob
        bs_html = BeautifulSoup(rw_html, 'html.parser')
        
        if(deep == True):
            for a in bs_html.find_all('a', href=True):
                
                if(a == None):
                    continue
                
                url_part_string = root_url + a['href']
                
                # If invalid URL then skip adding the url
                if(check_valid_url(url_part_string) == False):
                    continue
                    
                url_part_string = make_url_useful(url_part_string)
                
                hash_code = hashlib.md5(url_part_string.encode())

                #Add to stack if part url is not already a part of the proc_map else ignore
                if((hash_code.hexdigest() in url_proc_map) == False):
                    url_proc_stack.append(url_part_string)
                    url_proc_map[hash_code.hexdigest()] = 0
                    
    # Save to files before returning
    with open('hash2url.json', 'w', encoding='utf-8') as outfile:
        json.dump(hash_to_url_map, outfile, ensure_ascii=False, indent=2)
        
    with open('hash2text.json', 'w', encoding='utf-8') as outfile:
        json.dump(hash_to_text_map, outfile, ensure_ascii=False, indent=2)    
                    
    return

In [154]:
# Seeding steps
seed_url = 'https://www.eenadu.net'
root_url = ''
md5_encode = hashlib.md5(seed_url.encode())

url_process_map = {}
# Value of 0 indicates not processed, 1 indicates processed
url_process_map[md5_encode.hexdigest()] = 0

# Keep adding and popping from this list as you get more urls
url_process_stack = []
url_process_stack.append(seed_url)

In [155]:
# Create a black list of urls which return None when called on Beautiful Soup
# Just add the blacklisted urls to the url_process_map and do not add them to the stack. It is sufficient enough 
# condition for the blacklist

urls = []

for u in urls:
    md5_encode = hashlib.md5(u.encode())
    url_process_map[md5_encode.hexdigest()] = 1

In [156]:
crawl_scrape(url_process_map, url_process_stack, root_url)

==========Reached crawl limit.


In [157]:
# Printing the text - Just for illustration purposes
print(hash_to_text_map)

{'5b43d3ef350a0d35c31ad383f61edcea': 'కావల్సినవి: చామ దుంపలు - పావు కేజీ, బియ్యప్పిండి - పెద్ద చెంచా, కారం - చెంచా, పసుపు - పావుచెంచా, ఆమ్\u200cచూర్\u200c పొడి - అర చెంచా, ఉప్పు - తగినంత, నూనె - వేయించేందుకు సరిపడా.\xa0తాలింపు కోసం:మినప్పప్పు - చెంచా, ఆవాలు - అర చెంచా, ఎండుమిర్చి - రెండు, కరివేపాకు - రెబ్బ, వెల్లుల్లి రెబ్బలు - ఐదు, నూనె - చెంచా, కూరకారం - చెంచా.తయారీ: శుభ్రంగా కడిగిన చామదుంపల్ని కుక్కర్\u200cలో వేసి ఒక కూత వచ్చేవరకూ ఉడికించుకుని తీసుకోవాలి. తరవాత చెక్కు తీసి చక్రాల్లా కోయాలి. వీటిపై బియ్యప్పిండి, కారం, పసుపు, ఆమ్\u200cచూర్\u200c పొడి, ఉప్పు, నూనె వేసి ఇవన్నీ ముక్కలకు పట్టేలా బాగా కలపాలి. కావాలంటే బియ్యంపిండిని మరికొంచెం కూడా కలుపుకోవచ్చు. బాణలిని పొయ్యిమీద పెట్టి నూనె వేసి ఈ ముక్కల్ని కరకరలాడేలా వేయించి తీసుకోవాలి. ఇప్పుడు తాలింపు వేసుకోవాలి. బాణలిలో నూనె వేడిచేసి మెత్తగా దంచిన వెల్లుల్లి రెబ్బలు, ఆవాలు వేయాలి. అవి చిటపటలాడాక మినప్పప్పు, ఎండుమిర్చి, కరివేపాకు వేయాలి. దీంట్లో ఇందాక వేయించి పెట్టుకున్న చామ దుంప ముక్కలు వేసి, పైన కూరకారం చల్లాలి. రెండు నిమిషాలయ్యాక దింపే